https://docs.mapineq.org/api_tutorial_python

SCI is not avaaible on the website yet? 
can requet from pyckage, but cqan't do it from API ??

In [1]:
# Required for API queries
import requests
import json

# Required for data wrangling
import pandas as pd

BASE = "https://api.mapineq.org"

# Find Data by NUTS Region (get_source_by_nuts_level)

In [2]:
# Identify the API endpoint to query
endpoint = "functions/postgisftw.get_source_by_nuts_level/items.json"

# Provide parameters required by the endpoint
params = {
  "_level": 0,  # define the geographic level (i.e. NUTS level)
  "limit": 500  # limit response to this many data sets
}

# Submit HTTP GET request to query the API
response = requests.get(f"{BASE}/{endpoint}", params=params)

if response.status_code == 200:
    print(response)
else:
    raise Exception(f"Error: {response.status_code}")

# Convert JSON to DataFrame
data = response.json()
df_content = pd.DataFrame(data)

# View data
df_content

<Response [200]>


,f_description,f_resource,f_short_description
0,"Academic Budget by, year, NUTS 1, NUTS 2 and N...",eheso_budgetclass,Acad. Budget
1,"Academic Expenditure by, year, NUTS 1, NUTS 2 ...",eheso_expendclass,Acad. Expenditure
2,"Academic Funding by, year, NUTS 1, NUTS 2 and ...",eheso_fundingclass,Acad. Funding
3,"Academic Graduates by year, citizenship, level...",eheso_graduates,Acad. Graduates
4,"Acquisition of citizenship by age group, sex a...",MIGR_ACQ1CTZ,New citizenship
...,...,...,...
495,Regional disparities in the rate of young peop...,LFST_R_EDATD_22,NEET rates
496,Regional Economy (OECD),REGION_ECONOM,Regional Economy
497,Regional Innovation (OECD),REGION_INNOVATION,Regional Innovation
498,Regional Labour (OECD),REGION_LABOUR,Regional Labour


## Choose the dataset you are interetsed in 

In [3]:
df_content[df_content['f_resource'].str.contains('meta', case=False, na=False)]
# df_content[df_content['f_resource'].str.contains('gee', case=False, na=False)]
# df_content[df_content['f_resource'] == 'meta_sci']

,f_description,f_resource,f_short_description
332,META estimation of migration inflow at the cou...,meta_migration_in,META Migration In
333,META estimation of migration outflow at the co...,meta_migration_out,META Migration Out


## Structural filters
This is the same structure across all different dataset. 

In [4]:
import requests, pandas as pd, json

RESOURCE = "meta_migration_in"   # or "meta_migration_out" etc.

# 1. Check coverage: which years + levels are valid
resp = requests.get(
    f"{BASE}/functions/postgisftw.get_year_nuts_level_from_source/items.json",
    params={"_resource": RESOURCE},
    timeout=60
)
resp.raise_for_status()
coverage = pd.DataFrame(resp.json())
coverage = coverage.rename(columns={"f_year": "year", "f_level": "level"})
print("Coverage:\n", coverage.head())

Coverage:
   level  year
0     0  2019
1     0  2020
2     0  2021
3     0  2022


In [5]:
# Pick the most recent year + lowest level (0 = country)
year = int(coverage["year"].max())
level = int(coverage["level"].min())

print(f"Using year={year}, level={level}")

Using year=2022, level=0


## Attributes of the dataset
This is dataset specific

In [6]:
sel = {"year": str(year), "level": str(level), "selected": []}
resp = requests.get(
    f"{BASE}/functions/postgisftw.get_column_values_source_json/items.json",
    params={"_resource": RESOURCE, "source_selections": json.dumps(sel)},
    timeout=60
)
resp.raise_for_status()
filters = pd.DataFrame(resp.json())
filters

,field,field_label,field_values
0,origin,origin,"[{'label': 'RU', 'value': 'RU'}, {'label': 'DK..."
1,month,month,"[{'label': 'January', 'value': 'January'}, {'l..."
2,time_granularity,time_granularity,"[{'label': 'Annual', 'value': 'Annual'}, {'lab..."


In [7]:
conditions = [
    {"field": "origin", "value": "UK"},
    {"field": "month", "value": 'January'}   # None = don’t filter, but include month dimension
]

payload = {"source": RESOURCE, "conditions": conditions}
resp = requests.get(
    f"{BASE}/functions/postgisftw.get_x_data/items.json",
    params={"_level": level, "_year": year, "X_JSON": json.dumps(payload), "limit": 5000},
    timeout=60
)
resp.raise_for_status()
df = pd.DataFrame(resp.json())
df.head()

,data_year,geo,geo_name,geo_source,geo_year,x
0,2022,AL,Shqip�ria,NUTS,2024,220.0
1,2022,AT,�sterreich,NUTS,2024,101.0
2,2022,BE,Belgique/Belgi�,NUTS,2024,177.0
3,2022,BG,Bulgaria,NUTS,2024,744.0
4,2022,CH,Schweiz/Suisse/Svizzera,NUTS,2024,218.0


In [11]:
import json, requests, pandas as pd

BASE = "https://api.mapineq.org"

def values_for(filters_df, field_name: str):
    """Return a list of {'label','value'} dicts for a given field."""
    out = []
    for _, row in filters_df.iterrows():
        if row.get("field") == field_name:
            for v in (row.get("field_values") or []):
                out.append({"label": v.get("label"), "value": v.get("value")})
    return out

# 1) Inspect available values
origin_vals = values_for(filters, "origin")
month_vals  = values_for(filters, "month")
gran_vals   = values_for(filters, "time_granularity")

# 2) Pick values (validate they exist)
origin = "UK"
assert any(v["value"] == origin for v in origin_vals), f"{origin=} not in filters."

monthly_value = next((v["value"] for v in gran_vals if str(v["value"]).lower() == "monthly"), None)
assert monthly_value is not None, "Monthly granularity not available for this dataset."

# # ---------- A) Single month (e.g., January) ----------
# jan_value = next((v["value"] for v in month_vals if str(v["value"]).lower() == "january"), None)
# assert jan_value is not None, "January not available in month options."

# conditions = [
#     {"field": "origin",            "value": origin},
#     {"field": "time_granularity",  "value": monthly_value},
#     {"field": "month",             "value": jan_value},   # one month
# ]

# payload = {"source": RESOURCE, "conditions": conditions}
# r = requests.get(
#     f"{BASE}/functions/postgisftw.get_x_data/items.json",
#     params={"_level": level, "_year": year, "X_JSON": json.dumps(payload), "limit": 5000},
#     timeout=60
# )
# r.raise_for_status()
# df_jan = pd.DataFrame(r.json())
# print("January sample:")
# display(df_jan.head())

# ---------- B) All months (loop and concat) ----------
frames = []
for m in month_vals:   # month_vals came from your filters
    conditions = [
        {"field": "geo", "value": "AL"},
        {"field": "time_granularity", "value": "Monthly"},
        {"field": "month", "value": "January"},  # specific month
        # {"field": "month", "value": m["value"]},  # specific month
    ]
    payload = {"source": RESOURCE, "conditions": conditions}
    r = requests.get(
        f"{BASE}/functions/postgisftw.get_x_data/items.json",
        params={"_level": level, "_year": year, "X_JSON": json.dumps(payload)},
        timeout=360
    )
    r.raise_for_status()
    df_m = pd.DataFrame(r.json())
    df_m["month"] = m["value"]   # add the month label back
    frames.append(df_m)

df_all_months = pd.concat(frames, ignore_index=True)
df_all_months

,data_year,geo,geo_name,geo_source,geo_year,x,month
0,2022,AL,Shqipëria,NUTS,2021,0,January
1,2022,AL,Shqipëria,NUTS,2021,0,January
2,2022,AL,Shqipëria,NUTS,2021,217,January
3,2022,AL,Shqipëria,NUTS,2021,0,January
4,2022,AL,Shqipëria,NUTS,2021,9,January
...,...,...,...,...,...,...,...
125,2022,AL,Shqipëria,NUTS,2021,10,All
126,2022,AL,Shqipëria,NUTS,2021,0,All
127,2022,AL,Shqipëria,NUTS,2021,0,All
128,2022,AL,Shqipëria,NUTS,2021,0,All
